In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pprint
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

import visualize

In [26]:
import torch
from ops import *
from kernels import *
from config import *

_input = torch.empty(GLOBAL_CONFIG.batch_size, GLOBAL_CONFIG.input_seq_len, dtype=GLOBAL_CONFIG.act_dtype)
llama_3_8b = Llama3_8B(use_flash_attention=True)
output, prefill, decode = llama_3_8b(_input, output_len=GLOBAL_CONFIG.output_seq_len)ㅋㅋ

TTFT: 956.6947879034522 ms
TPOT: 9.597377451467427 ms
Latency: 20.61212380850874 s
Throughput: 99.35899953960978 token/s


In [27]:
prefill

{'name': None,
 'inference_time': 0.9566947879034522,
 'ops': 280963271622656,
 'mem': 329944924160,
 'arith_intensity': 0,
 'performance': 0,
 'mem_bandwidth': 0,
 'bound': '',
 'children': [<ops.GEMM at 0x15fbd5ff0>,
  <ops.GEMM at 0x15f9c39d0>]}

In [31]:
def get_result(data: dict):
    result = copy.deepcopy(data)
    result["children"] = {}
    
    if data["children"]:
        for child in data["children"]:
            if not isinstance(child, dict):  # Model
                if isinstance(child, list):
                    result["children"][child[0].name] = get_result(child[0].result)
                else:
                    result["children"][child.name] = get_result(child.result)
            else:
                result["children"][child["name"]] = get_result(child)
    return result

In [33]:
ss = get_result(prefill)

In [35]:
pprint.pprint(ss)

{'arith_intensity': 0,
 'bound': '',
 'children': {None: {'arith_intensity': 1.75,
                     'bound': 'memory',
                     'children': {},
                     'inference_time': 0.0001387263338501292,
                     'mem': 268435456,
                     'mem_bandwidth': 1935000000000.0,
                     'name': None,
                     'ops': 469762048,
                     'performance': 3386250000000.0},
              'decoder.layer.0': {'arith_intensity': 0,
                                  'bound': '',
                                  'children': {'attention_norm': {'arith_intensity': 1.75,
                                                                  'bound': 'memory',
                                                                  'children': {},
                                                                  'inference_time': 0.0001387263338501292,
                                                                  'mem': 268435456,
    

In [ ]:
import json 
   
# Convert and write JSON object to file
with open("sample.json", "w") as outfile: 
    json.dump(ss, outfile)